UnZip Files from a location

In [1]:
import os,zipfile
dir_name = '/data1/call_miner/Vinyas_Call_miner/call_miner'
os.chdir(dir_name)
zip_ref = zipfile.ZipFile('Vail_CDR_Transcripts_Feb18.zip') # create zipfile object
zip_ref.extractall('/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_Unzipped') # extract file to dir
        #zip_ref.close() # close file

Extract nested zipped files 

In [2]:
import os,zipfile
dir_name = '/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_Unzipped'
extension = ".zip"

os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall('/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_xml_unzipped') #

Apply fix to xml

In [3]:
#os.chdir("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_xml_unzipped")
import os, fnmatch
def findReplace(directory, find, replace, filePattern):
    for path, dirs, files in os.walk(os.path.abspath(directory)):
        for filename in fnmatch.filter(files, filePattern):
            filepath = os.path.join(path, filename)
            with open(filepath) as f:
                s = f.read()
            s = s.replace(find, replace)
            with open(filepath, "w") as f:
                f.write(s)

findReplace("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_xml_unzipped", "</segment>\n<transcript>", "</segment>\n</transcript>", "*.xml")                
                

Extract Relavent Text from the XML

In [19]:
#Extract both Customer and Agent channel data

import unicodecsv as csv
import xml.etree.ElementTree as ET
import glob, os
import io

# Set the working directory
os.chdir("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_xml_unzipped")
for filename in glob.glob('*.xml'):
                 tree = ET.parse(filename,ET.XMLParser(encoding='Latin1'))
                 root=tree.getroot()
                 eureka_id=root.find('eureka_id').text
                 with open("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_Text_data/file_"+eureka_id+".txt","w") as f:
                        csvwriter = csv.writer(f)
                        for time in root.findall('segment'):
                            row=[]
                            task_name = time.find('words').text
                            row.append(task_name)
                            csvwriter.writerow(row)
                            
f.close()

In [4]:
#Extract only Customer channel data

import unicodecsv as csv
import xml.etree.ElementTree as ET
import glob, os
import io

# Set the working directory
os.chdir("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_xml_unzipped")
for filename in glob.glob('*.xml'):
                 tree = ET.parse(filename,ET.XMLParser(encoding='Latin1'))
                 root=tree.getroot()
                 eureka_id=root.find('eureka_id').text
                 with open("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_Cust_words/file_"+eureka_id+".txt","w") as f:
                        csvwriter = csv.writer(f)
                        for time in root.findall('segment'):
                            row=[]
                            if time.find('speaker').text=='Customer':
                                task_name = time.find('words').text
                                row.append(task_name)
                                csvwriter.writerow(row)
                            
f.close()

In [5]:
#Extract only Agent channel data

import unicodecsv as csv
import xml.etree.ElementTree as ET
import glob, os
import io

# Set the working directory
os.chdir("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_xml_unzipped")
for filename in glob.glob('*.xml'):
                 tree = ET.parse(filename,ET.XMLParser(encoding='Latin1'))
                 root=tree.getroot()
                 eureka_id=root.find('eureka_id').text
                 with open("/data1/call_miner/Vinyas_Call_miner/call_miner/Feb_Cust_words/file_"+eureka_id+".txt","w") as f:
                        csvwriter = csv.writer(f)
                        for time in root.findall('segment'):
                            row=[]
                            if time.find('speaker').text=='Agent':
                                task_name = time.find('words').text
                                row.append(task_name)
                                csvwriter.writerow(row)
                            
f.close()

In [7]:
pyLDAvis.enable_notebook()

In [3]:
import gensim

In [2]:
from gensim import corpora, models, similarities

/data1/ANACONDA/Anaconda2.7/lib/python2.7/site-packages/numpy/lib/utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


In [1]:
import pyLDAvis.gensim as gensimvis


In [9]:
import pyLDAvis.sklearn